In [1]:
import requests
import tensorflow as tf
import numpy as np
import os
from matplotlib.image import imread, imsave 
import matplotlib.pyplot as plt  
import pandas as pd
from konlpy.tag import Mecab
from gensim.models import word2vec
print("load done")

load done


# Parameters

In [2]:
vector_size = 50
encode_length = 6
label_size = 10
embed_type = "onehot" #onehot or w2v

# Vector model

In [3]:
def train_vector_model(path):
    with open (path, "r") as myfile : 
        str_buf = myfile.readlines()

    mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
    pos1 = mecab.pos(''.join(str_buf))
    pos2 = ' '.join(list(map(lambda x : '\n' if x[1] in ['SF'] else x[0], pos1))).split('\n')
    morphs = list(map(lambda x : mecab.morphs(x) , pos2))
    model = word2vec.Word2Vec(size=vector_size, window=2, min_count=1)
    model.build_vocab(morphs)
    model.train(morphs)
    return model

model = train_vector_model("./data/train.txt")

# Load Train Data

In [4]:
def load_csv(data_path):
    df_csv_read = pd.read_csv(data_path,
                              skipinitialspace=True,
                              engine="python",
                              encoding='utf-8-sig')
    return df_csv_read

# Embed word to vector

In [5]:
def embed(data) : 
    mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
    inputs = []
    labels = []
    
    for encode_raw in data['encode'] : 
        encode_raw = mecab.morphs(encode_raw)
        encode_raw = list(map(lambda x : encode_raw[x] if x < len(encode_raw) else '#', range(encode_length)))
        if(embed_type == 'onehot') :
            bucket = np.zeros(vector_size, dtype=float).copy()
            input = np.array(list(map(lambda x : onehot_vectorize(bucket, x) if x in model.wv.index2word else np.zeros(vector_size,dtype=float) , encode_raw)))
        else : 
            input = np.array(list(map(lambda x : model[x] if x in model.wv.index2word else np.zeros(vector_size,dtype=float) , encode_raw)))
     
        inputs.append(input.flatten())
        
    for decode_raw in data['decode'] : 
        label = np.zeros(label_size, dtype=float)
        np.put(label, decode_raw, 1)
        labels.append(label)
    
    return inputs, labels

def onehot_vectorize(bucket, x):
    np.put(bucket, model.wv.index2word.index(x),1)
    return bucket
    

# Embed word to vector on predict step

In [6]:
def inference_embed(data) : 
    mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
    encode_raw = mecab.morphs(data)
    encode_raw = list(map(lambda x : encode_raw[x] if x < len(encode_raw) else '#', range(encode_length)))
    if(embed_type == 'onehot') :
        bucket = np.zeros(vector_size, dtype=float).copy()
        input = np.array(list(map(lambda x : onehot_vectorize(bucket, x) if x in model.wv.index2word else np.zeros(vector_size,dtype=float) , encode_raw)))
    else : 
        input = np.array(list(map(lambda x : model[x] if x in model.wv.index2word else np.zeros(vector_size,dtype=float) , encode_raw)))
    return input

# get train and test data for feed on tensorflow session

In [7]:
def get_test_data() :
    train_data, train_label = embed(load_csv("./data/train.csv"))
    test_data, test_label = embed(load_csv("./data/test.csv"))
    return train_label, test_label, train_data, test_data

# create graph

In [8]:
def create_graph(train=True):
    # placeholder is used for feeding data.
    x = tf.placeholder("float", shape=[None, encode_length * vector_size], name = 'x') 
    y_target = tf.placeholder("float", shape=[None, label_size], name = 'y_target') 

    # reshape input data
    x_image = tf.reshape(x, [-1,encode_length,vector_size,1], name="x_image")
    
    # Build a convolutional layer and maxpooling with random initialization
    W_conv1 = tf.Variable(tf.truncated_normal([2, 2, 1, 32], stddev=0.1), name="W_conv1") # W is [row, col, channel, feature]
    b_conv1 = tf.Variable(tf.zeros([32]), name="b_conv1")
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1, name="h_conv1")
    h_pool1 = tf.nn.max_pool( h_conv1 , ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name = "h_pool1")
    
    # Build a fully connected layer
    h_pool2_flat = tf.reshape(h_pool1, [-1, int((encode_length/2)*(vector_size/2))*32], name="h_pool2_flat")
    W_fc1 = tf.Variable(tf.truncated_normal([int((encode_length/2)*(vector_size/2))*32, 256], stddev=0.1), name = 'W_fc1')
    b_fc1 = tf.Variable(tf.zeros([256]), name = 'b_fc1')
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name="h_fc1")
    
    keep_prob = 1.0
    if(train) : 
        # Dropout Layer
        keep_prob = tf.placeholder("float", name="keep_prob")
        h_fc1 = tf.nn.dropout(h_fc1, keep_prob, name="h_fc1_drop")
    
    # Build a fully connected layer with softmax 
    W_fc2 = tf.Variable(tf.truncated_normal([256, label_size], stddev=0.1), name = 'W_fc2')
    b_fc2 = tf.Variable(tf.zeros([label_size]), name = 'b_fc2')
    y=tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2, name="y")
    
    # define the Loss function
    #cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_target))
    
    # define optimization algorithm
    #train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
    # correct_prediction is list of boolean which is the result of comparing(model prediction , data)

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
    # tf.cast() : changes true -> 1 / false -> 0
    # tf.reduce_mean() : calculate the mean
    
    # create summary of parameters
    tf.summary.histogram('weights_1', W_conv1)
    tf.summary.histogram('y', y)
    tf.summary.scalar('cross_entropy', cross_entropy)
    merged = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter("/tmp/cnn")
    
    return accuracy, x, y_target, keep_prob, train_step, merged, y, cross_entropy, summary_writer, W_conv1
    
print("define cnn graph func")

define cnn graph func


# visualize weight matrix function 

In [9]:
def show_layer(weight_list) :
    for matrix in weight_list[0] :  
        print(plt.imshow(matrix))

# run train 

In [10]:
def run() : 
    try : 
        # get Data 
        labels_train, labels_test, data_filter_train, data_filter_test = get_test_data()
        # reset Graph
        tf.reset_default_graph()   
        # Create Session
        sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))  
        # create graph
        accuracy, x, y_target, keep_prob, train_step, merged, y, cross_entropy, summary_writer, W_conv1 = create_graph(train=True)
        # set saver
        saver = tf.train.Saver(tf.all_variables())
        # initialize the variables
        sess.run(tf.global_variables_initializer())
    
        # training the MLP
        for i in range(200): 
            sess.run(train_step, feed_dict={x: data_filter_train, y_target: labels_train, keep_prob: 0.5})
            if i%10 == 0:
                train_accuracy = sess.run(accuracy, feed_dict={x:data_filter_train, y_target: labels_train, keep_prob: 1})
                print ("step %d, training accuracy: %.3f"%(i, train_accuracy))
                
                # calculate the summary and write.
                summary = sess.run(merged, feed_dict={x:data_filter_train, y_target: labels_train, keep_prob: 1})
                summary_writer.add_summary(summary , i)
                print("summary for tensorboard written")
                
        # for given x, y_target data set
        print  ("test accuracy: %g"% sess.run(accuracy, feed_dict={x:data_filter_test, y_target: labels_test, keep_prob: 1}))
        
        # show weight matrix as image 
        weight_vectors = sess.run(W_conv1, feed_dict={x: data_filter_train, y_target: labels_train, keep_prob: 1.0})
        show_layer(weight_vectors)
        
        # Save Model
        path = './model/'
        if not os.path.exists(path):
            os.makedirs(path)
            print("path created")
        saver.save(sess, path)
        print("model saved")
    except Exception as e : 
        raise Exception ("error on training: {0}".format(e))
    finally :
        sess.close()

# run stuff
run()

Instructions for updating:
Please use tf.global_variables instead.
step 0, training accuracy: 0.118
summary for tensorboard written
step 10, training accuracy: 0.471
summary for tensorboard written
step 20, training accuracy: 0.765
summary for tensorboard written
step 30, training accuracy: 0.765
summary for tensorboard written
step 40, training accuracy: 0.765
summary for tensorboard written
step 50, training accuracy: 0.706
summary for tensorboard written
step 60, training accuracy: 0.765
summary for tensorboard written
step 70, training accuracy: 0.765
summary for tensorboard written
step 80, training accuracy: 0.882
summary for tensorboard written
step 90, training accuracy: 1.000
summary for tensorboard written
step 100, training accuracy: 1.000
summary for tensorboard written
step 110, training accuracy: 1.000
summary for tensorboard written
step 120, training accuracy: 1.000
summary for tensorboard written
step 130, training accuracy: 1.000
summary for tensorboard written
step 1

# predict test set

In [11]:
def predict(test_data) : 
    try : 
        # reset Graph
        tf.reset_default_graph()   
        # Create Session
        sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))  
        # create graph
        _, x, _, _, _, _, y, _, _, _ = create_graph(train=False)
        
        # initialize the variables
        sess.run(tf.global_variables_initializer())
        
        # set saver
        saver = tf.train.Saver()
        
        # Restore Model
        path = './model/'
        if os.path.exists(path):
            saver.restore(sess, path)
            print("model restored")

        # training the MLP
        #print("input data : {0}".format(test_data))
        y = sess.run([y], feed_dict={x: np.array([test_data])})
        print("result : {0}".format(y))
        print("result : {0}".format(np.argmax(y)))
        
    except Exception as e : 
        raise Exception ("error on training: {0}".format(e))
    finally :
        sess.close()

print("words in dict : {0}".format(model.wv.index2word))
# run stuff
predict(np.array(inference_embed("다시는 오지 않을 거야")).flatten())
predict(np.array(inference_embed("돈이 아깝지 않다")).flatten())

words in dict : ['다', '이', '정말', '맛', '긍정', '가', '부정', '별로', '다시', '적', '식당', '지', '아깝', '야', '.', '영화', '은', '음식', '있', '돈', '없', '않', '어', '꼭', '재미있', '겠어', '와야', '거', '을', '는', '였', '맛있', '오', '재미없']
INFO:tensorflow:Restoring parameters from ./model/
model restored
result : [array([[  3.39150688e-06,   9.93813753e-01,   6.11760747e-03,
          1.16159163e-05,   3.05861090e-06,   3.36837475e-05,
          2.23066650e-06,   7.18798628e-06,   5.37696997e-06,
          2.05518018e-06]], dtype=float32)]
result : 1
INFO:tensorflow:Restoring parameters from ./model/
model restored
result : [array([[  4.07658517e-05,   7.67132267e-02,   9.22763646e-01,
          8.06034659e-05,   5.87379909e-05,   2.00696159e-04,
          2.19792601e-05,   4.23768033e-05,   6.28341804e-05,
          1.50135611e-05]], dtype=float32)]
result : 2
